# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
    """,
)

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    In general, yes—bananas are a healthy, convenient fruit.
    
    Key points:
    - Nutrients: good source of potassium, vitamin B6, vitamin C, and dietary fiber.
    - Benefits: supports heart health, digestion, and steady energy.
    - Considerations: moderate sugar content; ripe bananas are sweeter. A medium banana (~105–110 kcal) has about 27 g carbs and 3 g fiber.
    - When to watch: if you have kidney disease or need to limit potassium, or if you’re watching blood sugar; portion size matters (1 medium is fine for most people).
    - Tips: choose ripe or slightly green depending on your energy needs; whole fruit > juice for fiber and fullness.
    
    Bottom line: a healthy, convenient part of a balanced diet.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [7]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are nutritious and healthy for most people.

Pros:
- Good potassium (helps blood pressure) and vitamin B6.
- Dietary fiber supports digestion.
- Contains vitamin C and some magnesium and antioxidants.
- Convenient, portable energy from natural sugars.

Considerations:
- Moderate in calories (~100 per medium fruit) and sugars; portion matters for diabetes or blood sugar management.
- Unripe bananas have more resistant starch; ripe ones are sweeter and higher in sugars.
- Very high potassium can be an issue for people with severe kidney disease.

Bottom line: for most people, a medium banana a day is a healthy part of a balanced diet.

_Good Job!_